In [259]:
import os
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('max_columns', 50)
pd.set_option('display.max_rows', 1000)

%matplotlib inline

In [233]:
def find_csv_filenames(
    path_to_dir, 
    suffix=".csv"
):
    '''
    read all data in path_to_dir with suffix type
    
    result:
      list with filenames  
    '''
    filenames = os.listdir(path_to_dir)
    out_files_lst = [ 
        filename for filename in filenames\
        if filename.endswith( suffix )]
    return out_files_lst


def get_date(
    file_name
):
    dt_ = file_name\
        .split('.')[0]
    return dt_.split('-')[2] + '-' +\
        dt_.split('-')[0] + '-' + \
        dt_.split('-')[1]
    

def concat_csse_d_rep_data(
    data_dir, 
    start_date,
    end_date
):
    df_lst = []
    
    # select all files between start and end dates
    data_dir_files =[
        f_ for f_ in find_csv_filenames(data_dir)\
            if (get_date(f_) >= start_date)
            and (get_date(f_) < end_date)
        ]
        
    for file_date in data_dir_files:

        df_day = pd.read_csv(
            os.path.join(
                data_dir,
                file_date
            )
        )\
        .assign(date=get_date(file_date))
        df_lst.append(df_day)
    
    return pd.concat(df_lst, axis=0, sort=False)

----

In [3]:
# set csse dirs:
csse_19_d_rep_dir =\
    'data/csse_covid_19_data/csse_covid_19_daily_reports/'
    
csse_19_ts_dir =\
    'data/csse_covid_19_data/csse_covid_19_time_series' 

csse_19_uid_dir =\
    'data/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'    

In [247]:
csse_d_rep_files_lst =\
    find_csv_filenames(csse_19_d_rep_dir)
    
csse_ts_files_lst =\
    find_csv_filenames(csse_19_ts_dir)
    
print(f'Cnt csse daily report data: {len(csse_d_rep_files_lst)}')
print(f'Cnt csse ts data: {len(csse_ts_files_lst)}')

Cnt csse daily report data: 79
Cnt csse ts data: 5


----

##### Concat all csse 19_daily-reports data:

In [ ]:
# 03-01-2020.csv: 8 colmun
# 03-22-2020.csv: 12 colnum

df_csse_d_rep_df_part1 = pd.read_csv(
        os.path.join(
            csse_19_d_rep_dir,
            csse_d_rep_files_lst[0]
        )
    )

df_csse_d_rep_df_part2 = pd.read_csv(
        os.path.join(
            csse_19_d_rep_dir,
            csse_d_rep_files_lst[50]
        )
    )

df_csse_d_rep_df_part3 = pd.read_csv(
        os.path.join(
            csse_19_d_rep_dir,
            csse_d_rep_files_lst[70]
        )
    )

print(df_csse_d_rep_df_part1.shape)
print(df_csse_d_rep_df_part1.head(2))

print(df_csse_d_rep_df_part2.shape)
print(df_csse_d_rep_df_part2.head(2))

print(df_csse_d_rep_df_part3.shape)
print(df_csse_d_rep_df_part3.head(2))

In [234]:
df_csse_d_rep_1 = concat_csse_d_rep_data(
    csse_19_d_rep_dir,
    start_date='2020-01-01',
    end_date='2020-03-22'
)

print(df_csse_d_rep_1.shape)
df_csse_d_rep_1.head(2)

(7617, 9)


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,date,Latitude,Longitude
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,2020-01-22,NaN,NaN


In [235]:
df_csse_d_rep_2 = concat_csse_d_rep_data(
    csse_19_d_rep_dir,
    start_date='2020-03-01',
    end_date='2020-03-22'
)

print(df_csse_d_rep_2.shape)
df_csse_d_rep_2.head(2)

(4799, 9)


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,date
0,Hubei,Mainland China,2020-03-01T10:13:19,66907,2761,31536,30.9756,112.2707,2020-03-01
1,NaN,South Korea,2020-03-01T23:43:03,3736,17,30,36.0000,128.0000,2020-03-01


In [236]:
df_csse_d_rep_3 = concat_csse_d_rep_data(
    csse_19_d_rep_dir,
    start_date='2020-03-22',
    end_date='2020-05-01'
)

print(df_csse_d_rep_3.shape)
df_csse_d_rep_3.head(2)

(57838, 13)


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,date
0,36061.0,New York City,New York,US,3/22/20 23:45,40.767273,-73.971526,9654,63,0,0,"New York City, New York, US",2020-03-22
1,36059.0,Nassau,New York,US,3/22/20 23:45,40.740665,-73.589419,1900,4,0,0,"Nassau, New York, US",2020-03-22


In [242]:
df_concat_12 = pd.concat([
    df_csse_d_rep_1\
    .assign(
        Latitude=np.nan,
        Longitude=np.nan,
        data_tp='1'
    ),
    df_csse_d_rep_2\
        .assign(
            data_tp='2'
        )
], axis=0, sort=False)

df_csse_d_rep = pd.concat([
    df_concat_12\
    .assign(
        Combined_Key='NAN',
        Admin2='NAN',
        FIPS=np.nan,
        Active=np.nan,
    )\
    .rename(columns={
        'Country/Region': 'Country_Region',
        'Province/State': 'Province_State',
        'Last Update': 'Last_Update',
        'Latitude': 'Lat',
        'Longitude': 'Lon'
    }),
    df_csse_d_rep_3\
    .assign(
        data_tp='3'
    )\
    .rename(columns={
        'Long_': 'Lon'
    })
], axis=0, sort=False)

del df_concat_12;
del df_csse_d_rep_1, df_csse_d_rep_2, df_csse_d_rep_3;

In [243]:
print(df_csse_d_rep.shape)
df_csse_d_rep.head(2)

(70254, 14)


,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,date,Lat,Lon,data_tp,Combined_Key,Admin2,FIPS,Active
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22,NaN,NaN,1,NAN,NAN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,2020-01-22,NaN,NaN,1,NAN,NAN,NaN,NaN


---

#### Load csse covid_19 time_series

In [249]:
csse_19_ts_dir

'data/csse_covid_19_data/csse_covid_19_time_series'

In [263]:
csse_ts_files_lst

['time_series_covid19_confirmed_global.csv',
 'time_series_covid19_confirmed_US.csv',
 'time_series_covid19_deaths_global.csv',
 'time_series_covid19_deaths_US.csv',
 'time_series_covid19_recovered_global.csv']

In [252]:
i = 0
pd.read_csv(
            os.path.join(
                csse_19_ts_dir,
                csse_ts_files_lst[i]
            )
        ).head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409


In [253]:
i = 1
pd.read_csv(
            os.path.join(
                csse_19_ts_dir,
                csse_ts_files_lst[i]
            )
        ).head(2)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,3,5,12,14,15,27,29,32,37,45,51,55,56,58,69,77,82,84,93,112,113,121,121,128


In [264]:
i = 2
pd.read_csv(
            os.path.join(
                csse_19_ts_dir,
                csse_ts_files_lst[i]
            )
        ).head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,2,4,4,4,4,4,4,4,6,6,7,7,11,14,14,15
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,2,2,2,2,4,5,5,6,8,10,10,11,15,15,16,17,20,20,21,22,22,23


In [261]:
i = 4
pd.read_csv(
            os.path.join(
                csse_19_ts_dir,
                csse_ts_files_lst[i]
            )
        ).head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,5,5,10,10,10,15,18,18,29,32
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,10,17,17,31,31,33,44,52,67,76,89,99,104,116,131,154,165


In [250]:
csse_ts_data_lst = []
for f_ in csse_ts_files_lst:
    csse_ts_data_lst.append(
        pd.read_csv(
            os.path.join(
                csse_19_ts_dir,
                f_
            )
        )
    )

In [ ]:
# for df_ in csse_ts_data_lst:
#     print(df_.shape, '\n')
#     print(df_.head(2))

-----

In [258]:
pd.read_csv(csse_19_uid_dir)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.939110,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.153300,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.033900,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.506300,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.202700,17.873900,Angola,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3566,84056037,US,USA,840.0,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",42343.0
3567,84056039,US,USA,840.0,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464.0
3568,84056041,US,USA,840.0,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226.0
3569,84056043,US,USA,840.0,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805.0
